In [1]:
import requests
import re
import time
import pandas as pd
from elasticsearch import Elasticsearch
from datetime import datetime

In [2]:
urls=["https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2016%2C2015%2C2014%2C2013%2C2012&r=788&p=all&rg=2&cc=150910&uitoken=4c599ed9273b53b875f870451acdf719","https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2016%2C2015&r=381&p=all&rg=2&cc=150910&uitoken=4c599ed9273b53b875f870451acdf719","https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2014%2C2013&r=381&p=all&rg=2&cc=150910&uitoken=4c599ed9273b53b875f870451acdf719","https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2012&r=381&p=all&rg=2&cc=150910&uitoken=4c599ed9273b53b875f870451acdf719","https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2016%2C2015%2C2014&r=724&p=all&rg=2&cc=150910&uitoken=4c599ed9273b53b875f870451acdf719",
     "https://comtrade.un.org/api/get?max=500&type=C&freq=A&px=HS&ps=2013%2C2012&r=724&p=all&rg=2&cc=150910&uitoken=4c599ed9273b53b875f870451acdf719"]

In [3]:
data_olive=[]
product='olive oil'

In [4]:
def get_content(url):
    url_content=requests.get(url) 
    return url_content.json()

In [6]:
elastic_connection = Elasticsearch('localhost:9200', timeout=3000.0)
counter=0
for url in urls: 
    json_text=get_content(url)
    data=json_text['dataset']
    for i in range (len(data)):
        years=str(data[i]['yr'])
        reporters=data[i]['rtTitle']
        partners=data[i]['ptTitle']
        trade_values=data[i]['TradeValue']
        item={"partners":partners,
                "reporters":reporters, 
                "years":years, 
                "trade_value":trade_values,
                "products":product,
                "retailers" :  "comtrade"}
        data_olive.append(item)
        doc_id = "%s-%s-%s"%('comtrade.un.org',time.time(),counter)
        counter+=1
        elastic_connection.index(index='trade',doc_type='export',id= doc_id, body=item)
    time.sleep(1)

In [7]:
df=pd.DataFrame(data_olive)

In [8]:
df

,partners,products,reporters,retailers,trade_value,years
0,World,olive oil,Tunisia,comtrade,329118171,2012
1,Algeria,olive oil,Tunisia,comtrade,137870,2012
2,Australia,olive oil,Tunisia,comtrade,172174,2012
3,Austria,olive oil,Tunisia,comtrade,85967,2012
4,Bahrain,olive oil,Tunisia,comtrade,211216,2012
5,Belgium,olive oil,Tunisia,comtrade,179975,2012
6,Brazil,olive oil,Tunisia,comtrade,52243,2012
7,Canada,olive oil,Tunisia,comtrade,3699378,2012
8,China,olive oil,Tunisia,comtrade,3987236,2012
9,Congo,olive oil,Tunisia,comtrade,81165,2012


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1914 entries, 0 to 1913
Data columns (total 5 columns):
partners        1914 non-null object
product         1914 non-null object
reporters       1914 non-null object
trade_values    1914 non-null int64
years           1914 non-null int64
dtypes: int64(2), object(3)
memory usage: 74.8+ KB


In [9]:
df.to_csv("olive_oil_data.csv")